In [1]:
import pandas as pd

# Load dataset
data = pd.read_csv("data.csv")
old_min = df['Probability'].min()
old_max = df['Probability'].max()

new_min = 50
new_max = 95

df['Probability_scaled'] = ((df['Probability'] - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

# Preview
print(data.head())
print("\nShape:", data.shape)
print("\nColumns:", data.columns)
print("\nInfo:")
print(data.info())


   Tenth  Twelfth  CGPA Branch  Internships  Projects  Aptitude  Soft skills  \
0  79.97    60.04   NaN    CSE            1         2     62.82        22.40   
1  73.62    77.76  6.87  CIVIL            1         0     42.10        73.96   
2  81.48    85.22  8.15     IT            1         5     56.08        54.59   
3  90.23    81.31  7.07     IT            1         2     71.85        89.45   
4  72.66    84.93  7.94     IT            1         2     58.22        82.04   

   Leadership  Extracurricular  Certifications  Backlogs    DSA  Lang  \
0       64.73            41.01               2         1  31.82     2   
1       62.72            17.85               1         0  59.50     3   
2       69.44            62.06               1         1  65.57     2   
3       69.47            44.74               0         1  26.38     3   
4       74.46            52.91               2         0  66.35     1   

   Probability  
0         46.4  
1         52.7  
2         60.4  
3         58

In [3]:
X = data.drop("Probability", axis=1)   # replace with your actual target column
y = data["Probability"]


In [5]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Identify categorical & numeric columns
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

# Preprocessing: OneHot for categorical + Scaling for numeric
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])


In [11]:
counts = y.value_counts()
rare_classes = counts[counts < 2].index
mask = ~y.isin(rare_classes)

X = X[mask]
y = y[mask]


In [12]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, test_idx in split.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]



In [13]:
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.